In [1]:
%pip install PyQt5
%pip install --upgrade replicate
%pip install openai requests PyQt5



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the API tokens
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Create or update the .env file with the API tokens
with open('.env', 'w') as env_file:
    env_file.write(f"REPLICATE_API_TOKEN={REPLICATE_API_TOKEN}\n")
    env_file.write(f"OPENAI_API_KEY={OPENAI_API_KEY}\n")

# Now you can use os.environ to get the API tokens
replicate_token = os.environ.get("REPLICATE_API_TOKEN")
openai_key = os.environ.get("OPENAI_API_KEY")  # Get OpenAI key from environment

from openai import OpenAI
client = OpenAI(api_key=openai_key)  # Use the retrieved OpenAI key
   
print(f"Replicate API Token: {replicate_token}")
print(f"OpenAI API Key: {openai_key}")  # Now using the defined openai_key variable


Replicate API Token: r8_6GvXzUzJXnqyVxYl4CRv1yf9MYiIz1r4CIiBR
OpenAI API Key: sk-bfmrifNNIY7ODXiDgy8mtEOzID-VLIxmfQOa5ofNwrT3BlbkFJn4woy_EVbIPIqVH6rer8reH5CBHwYfkSo8XEtj4ucA


In [1]:
import sys
import random
import os
import replicate
from replicate.client import Client
import requests
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QLabel,
    QScrollArea, QHBoxLayout, QMessageBox, QGridLayout, QMainWindow, QFrame,
    QTabWidget, QTextEdit, QSplitter
)
from PyQt5.QtCore import Qt, QTimer, QPropertyAnimation, QRect, pyqtProperty, QSize, QThread, pyqtSignal
from PyQt5.QtGui import QClipboard, QPixmap, QPainter, QIcon, QTransform, QColor, QFont
from PyQt5.QtWidgets import QGraphicsOpacityEffect
import logging
from openai import OpenAI


# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Replicate client
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
if not REPLICATE_API_TOKEN:
    logging.error("REPLICATE_API_TOKEN is not set.")
    raise SystemExit("Error: REPLICATE_API_TOKEN is not set.")

replicate_client = ReplicateClient(api_token=REPLICATE_API_TOKEN)

# Initialize OpenAI client
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    logging.error("OPENAI_API_KEY is not set.")
    raise SystemExit("Error: OPENAI_API_KEY is not set.")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Test connections
try:
    # Test Replicate connection
    replicate_client.run(
        "stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf",
        input={"prompt": "test"}
    )
    logging.info("Successfully connected to Replicate")
except Exception as e:
    logging.error(f"Error connecting to Replicate: {str(e)}")
    logging.error("Failed to connect to Replicate. The application may not function correctly.")

try:
    # Test OpenAI connection
    openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Hello!"}]
    )
    logging.info("Successfully connected to OpenAI")
except Exception as e:
    logging.error(f"Error connecting to OpenAI: {str(e)}")
    logging.error("Failed to connect to OpenAI. The application may not function correctly.")
    
# Toast Notification Class
class Toast(QWidget):
    def __init__(self, message, parent=None):
        super().__init__(parent)
        self.setWindowFlags(Qt.FramelessWindowHint | Qt.WindowStaysOnTopHint | Qt.Tool)
        self.setAttribute(Qt.WA_TranslucentBackground)
        self.opacity_effect = QGraphicsOpacityEffect(self)
        self.setGraphicsEffect(self.opacity_effect)
        self.opacity_effect.setOpacity(0)
        self.label = QLabel(message, self)
        self.label.setStyleSheet("""
            QLabel {
                background-color: rgba(0, 0, 0, 180);
                color: white;
                border-radius: 10px;
                padding: 10px;
            }
        """)
        font = QFont()
        font.setPointSize(10)
        self.label.setFont(font)
        self.animation = QPropertyAnimation(self.opacity_effect, b"opacity")
        self.animation.setDuration(2000)
        self.animation.setStartValue(0)
        self.animation.setKeyValueAt(0.1, 1)
        self.animation.setKeyValueAt(0.9, 1)
        self.animation.setEndValue(0)
        self.animation.finished.connect(self.close)

    def show_toast(self):
        self.show()
        self.animation.start()

# Worker Thread for Image Generation
class ImageGeneratorThread(QThread):
    finished = pyqtSignal(str)  # Signal to emit the image URL

    def __init__(self, prompt):
        super().__init__()
        self.prompt = prompt

    def run(self):
        try:
            # Initialize Replicate with the API token
            client = Client(api_token=REPLICATE_API_TOKEN)
            # Run the model
            output = client.run(
                "black-forest-labs/flux-pro",
                input={"prompt": self.prompt}
            )
            # Since the output is a direct URL, emit it
            self.finished.emit(output)
        except Exception as e:
            self.finished.emit(f"Error: {str(e)}")

# Custom Rounded Label
class RoundedClickableLabel(QLabel):
    clicked = pyqtSignal()

    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setStyleSheet("""
            QLabel {
                background-color: #000000;
                border: 2px solid #ffffff;
                border-radius: 15px;
                padding: 15px;
                color: #ffffff;
            }
            QLabel:hover {
                background-color: #333333;
            }
        """)
        self.setWordWrap(True)
        self.setCursor(Qt.PointingHandCursor)
        self.setFont(QFont("Arial", 10))

    def mousePressEvent(self, event):
        self.clicked.emit()

# Randomize Button with Spinning Logo
class RandomizeButton(QPushButton):
    def __init__(self, parent=None):
        super().__init__("Randomize", parent)
        self.setFixedSize(150, 60)
        self.setStyleSheet("""
            QPushButton {
                background-color: #4CAF50;
                border-radius: 30px;
                color: white;
                font-weight: bold;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
        """)
        self.spinning = False
        self.angle = 0
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_rotation)

    def paintEvent(self, event):
        super().paintEvent(event)
        if self.spinning:
            painter = QPainter(self)
            painter.setRenderHint(QPainter.Antialiasing)
            painter.translate(self.width() / 2, self.height() / 2)
            painter.rotate(self.angle)
            painter.drawPixmap(-15, -15, QPixmap("path/to/your/spinning_icon.png").scaled(30, 30, Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def start_spinning(self):
        self.spinning = True
        self.setText("")
        self.timer.start(50)  # Update every 50ms

    def stop_spinning(self):
        self.spinning = False
        self.setText("Randomize")
        self.timer.stop()

    def update_rotation(self):
        self.angle += 10
        if self.angle >= 360:
            self.angle = 0
        self.update()

# Main Application Window
class MainWindow(QMainWindow):
    def __init__(self, prompts):
        super().__init__()
        self.setWindowTitle("AI Society Showcase")
        self.setGeometry(100, 100, 1000, 800)
        self.prompts = prompts

        # Initialize Clipboard
        self.clipboard = QApplication.clipboard()

        # Central Widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)
        main_layout.setContentsMargins(0, 0, 0, 0)
        main_layout.setSpacing(0)

        # Tabs
        self.tabs = QTabWidget()
        main_layout.addWidget(self.tabs)

        # Prompts Tab
        self.prompts_tab = QWidget()
        self.tabs.addTab(self.prompts_tab, "Prompts")
        self.init_prompts_tab()

        # Image Generation Tab
        self.image_tab = QWidget()
        self.tabs.addTab(self.image_tab, "Image Generation")
        self.init_image_tab()

        # Set Stylesheet
        self.setStyleSheet("""
            QMainWindow {
                background-color: #000000;
            }
            QLabel {
                color: #ffffff;
            }
            QPushButton {
                background-color: #333333;
                color: #ffffff;
                border: none;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #555555;
            }
            QTextEdit {
                background-color: #111111;
                color: #ffffff;
            }
            QTabWidget::pane {
                border: none;
            }
            QTabBar::tab {
                background: #333333;
                color: #ffffff;
                padding: 10px;
            }
            QTabBar::tab:selected {
                background: #555555;
            }
            QScrollArea {
                background-color: #000000;
            }
        """)

    def init_prompts_tab(self):
        layout = QVBoxLayout(self.prompts_tab)
        
        # Add randomize button
        self.randomize_button = RandomizeButton()
        self.randomize_button.clicked.connect(self.randomize_prompt)
        layout.addWidget(self.randomize_button, alignment=Qt.AlignCenter)
        
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        layout.addWidget(scroll_area)

        container = QWidget()
        scroll_area.setWidget(container)
        self.container_layout = QVBoxLayout(container)
        self.container_layout.setAlignment(Qt.AlignTop)

        self.update_prompts_list()

    def update_prompts_list(self):
        # Clear existing prompts
        for i in reversed(range(self.container_layout.count())): 
            self.container_layout.itemAt(i).widget().setParent(None)

        for prompt in self.prompts:
            prompt_label = RoundedClickableLabel(prompt)
            prompt_label.clicked.connect(lambda checked, p=prompt: self.copy_to_clipboard(p))
            self.container_layout.addWidget(prompt_label)
            self.container_layout.addSpacing(10)

    def randomize_prompt(self):
        if self.prompts:
            self.randomize_button.start_spinning()
            QTimer.singleShot(3000, self.finish_randomize)  # 3 seconds of spinning

    def finish_randomize(self):
        self.randomize_button.stop_spinning()
        random_prompt = random.choice(self.prompts)
        self.copy_to_clipboard(random_prompt)
        self.show_toast("Random prompt copied to clipboard!")

    def init_image_tab(self):
        layout = QVBoxLayout(self.image_tab)

        # Splitter to adjust sizes
        splitter = QSplitter(Qt.Vertical)
        layout.addWidget(splitter)

        # Image Display Label (larger portion)
        self.image_label = QLabel("Your generated image will appear here.")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("background-color: #111111; color: #ffffff;")
        self.image_label.setMinimumHeight(400)
        splitter.addWidget(self.image_label)

        # Prompt Input Area (smaller portion)
        prompt_container = QWidget()
        prompt_layout = QVBoxLayout(prompt_container)
        prompt_layout.setContentsMargins(10, 10, 10, 10)

        self.prompt_edit = QTextEdit()
        self.prompt_edit.setPlaceholderText("Enter your prompt here")
        self.prompt_edit.setFixedHeight(100)
        prompt_layout.addWidget(self.prompt_edit)

        self.generate_button = QPushButton("Generate Image")
        self.generate_button.clicked.connect(self.on_generate_clicked)
        prompt_layout.addWidget(self.generate_button)

        splitter.addWidget(prompt_container)

        splitter.setStretchFactor(0, 3)  # Image label takes 3 parts
        splitter.setStretchFactor(1, 1)  # Prompt input takes 1 part

    def on_generate_clicked(self):
        prompt = self.prompt_edit.toPlainText().strip()
        if not prompt:
            QMessageBox.warning(self, "Warning", "Please enter a prompt.")
            return

        # Disable the button to prevent multiple clicks
        self.generate_button.setEnabled(False)
        self.image_label.setText("Generating Image...")

        # Start the image generation in a separate thread
        self.image_thread = ImageGeneratorThread(prompt)
        self.image_thread.finished.connect(self.on_image_generated)
        self.image_thread.start()

    def copy_to_clipboard(self, text):
        self.clipboard.setText(text)
        self.show_toast("Prompt copied to clipboard!")
        # Automatically switch to the Image Generation tab
        self.tabs.setCurrentWidget(self.image_tab)
        self.prompt_edit.setText(text)

    def show_toast(self, message):
        toast = Toast(message, self)
        toast.move(self.width() // 2 - toast.width() // 2, 50)
        toast.show_toast()

    def on_image_generated(self, output):
        self.generate_button.setEnabled(True)
        if output.startswith("Error"):
            self.image_label.setText("Failed to generate image.")
            self.show_toast(output)
        else:
            # Download the image and display it
            try:
                response = requests.get(output)
                if response.status_code == 200:
                    image = QPixmap()
                    image.loadFromData(response.content)
                    scaled_image = image.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    self.image_label.setPixmap(scaled_image)
                    self.image_label.setText("")
                else:
                    self.image_label.setText("Failed to load image.")
                    self.show_toast("Failed to load generated image.")
            except Exception as e:
                self.image_label.setText("Error loading image.")
                self.show_toast(str(e))

# Entry Point
if __name__ == "__main__":
    # Ensure the Replicate API token is set
    if not REPLICATE_API_TOKEN:
        print("Error: REPLICATE_API_TOKEN is not set.")
        sys.exit(1)

    prompts = [
        "futuristic 8K HD poster design, AI Showcase event, sleek modern cityscape background, glowing neon accents, circuit patterns, large abstract AI brain foreground, bright blue purple pink lights, intricate interconnected nodes and wires, floating holographic elements, streams of digital data, 3D geometric shapes, dynamic high-tech atmosphere, clean modern vibrant style, sharp detailed features, space at top for event text information, high quality, ultra-realistic rendering, immersive depth and perspective",
        "surreal 8K HD dreamscape, vibrant and contrasting colors, floating islands with lush greenery and cascading waterfalls, whimsical and mythical creatures interacting, twilight sky filled with twinkling stars and colorful nebulae, reflective and crystal-clear lakes, intricate and ornate floral patterns, soft glowing ambient lights, magical and enchanting atmosphere, high-resolution fantasy art, ultra-detailed textures, immersive and otherworldly environment, space for personalized text",
        "steampunk-inspired 8K HD machinery scene, highly detailed gears and cogs in motion, vintage brass and copper color palette, ornate and intricate metalwork, Victorian-era aesthetics with modern twists, steam-powered robots performing tasks, complex industrial background with pipes and steam vents, sepia tones with vibrant highlights, high-detail illustration with depth, dynamic and action-packed composition, ultra-realistic textures, space for event information",
        "abstract geometric 8K HD patterns, bold and vibrant colors with high contrast, symmetrical and intricate designs, interlocking and overlapping shapes creating depth, 3D effects with shadows and highlights, modern and contemporary art style, smooth vibrant gradients blending seamlessly, precise and intricate line work, high-resolution digital art with sharp clarity, visually striking and balanced composition, ultra-detailed elements, space for custom text overlays",
        "cyberpunk 8K HD street scene, neon-lit alleyways with vibrant signage, rainy weather creating reflective surfaces, diverse and futuristic characters with unique styles, holographic advertisements floating in the air, high-tech gadgets and wearable tech accessories, dark and moody atmosphere with contrasting bright lights, detailed urban environment with towering skyscrapers, vibrant color contrasts with electric blues and hot pinks, ultra-realistic textures, space for event details",
        "fantasy dragon 8K HD soaring over a medieval castle, dramatic and dynamic sky with sunset hues blending into twilight, highly intricate dragon scales shimmering with vibrant colors, detailed and majestic castle architecture with towering spires, dynamic action pose capturing movement, rich and textured environments with lush landscapes, high-resolution fantasy illustration with depth, epic and majestic feel, space for descriptive text",
        "underwater utopia in 8K HD, vibrant and colorful coral reefs teeming with diverse marine life, sunlight filtering through crystal-clear water creating dynamic light patterns, futuristic underwater cities with sleek architecture, bioluminescent plants and creatures glowing softly, serene and peaceful atmosphere with gentle currents, high-detail aquatic scene with realistic textures, vivid and saturated colors, ultra-realistic rendering, space for informational text",
        "space exploration 8K HD theme, highly detailed spaceship orbiting a distant and vibrant planet, starry background with vivid galaxies and nebulae, astronaut in a dynamic pose floating in zero gravity, vibrant celestial colors with rich textures, high-resolution sci-fi art with expansive depth, awe-inspiring and immersive view, ultra-realistic space elements, space for event title and details",
        "ancient mystical forest in 8K HD, towering and ancient trees with glowing bioluminescent leaves, magical and mythical creatures roaming the forest floor, misty and ethereal atmosphere with soft light beams piercing through the canopy, intricate details in flora and fauna with rich textures, serene and enchanting ambiance, high-resolution fantasy landscape with depth, ultra-realistic rendering, space for descriptive text",
        "modern minimalist 8K HD design, clean and sharp lines with precise geometry, monochromatic color scheme accented with a vibrant pop of color, abstract representation of advanced technology, high-resolution digital art with sleek and sophisticated aesthetic, spacious and uncluttered composition, ultra-detailed elements with subtle gradients, visually appealing and elegant, space for event information",
        "retro 80s synthwave 8K HD poster, vibrant and glowing neon colors with high saturation, grid landscape fading into a colorful sunset, futuristic and sleek retro cars speeding along the grid, pixelated and digital elements enhancing the nostalgic vibe, dynamic and energetic composition with motion effects, high-detail digital illustration with depth, stylish and eye-catching design, space for event title and details",
        "abstract expressionism 8K HD artwork, bold and dynamic brushstrokes with vibrant and contrasting colors, emotional and energetic composition conveying movement, high-resolution fine art style with rich textures, visually impactful and thought-provoking elements, ultra-detailed layers creating depth, modern and contemporary aesthetic, space for artistic text or titles",
        "biotech-inspired 8K HD visuals, DNA strands intricately intertwined with glowing circuitry, vibrant and pulsating light elements enhancing the futuristic feel, high-tech laboratory background with advanced equipment, fusion of organic and technological motifs, high-resolution and ultra-detailed design, vibrant and dynamic color palette, visually stunning and innovative, space for descriptive text",
        "magical library interior in 8K HD, endless rows of ancient bookshelves with floating and glowing books, warm and ambient lighting casting soft shadows, intricate architectural details with ornate decorations, whimsical and cozy atmosphere inviting exploration, high-resolution fantasy illustration with rich textures, enchanting and inviting environment, ultra-realistic rendering, space for event information",
        "post-apocalyptic 8K HD landscape, abandoned and decaying city overtaken by lush and wild nature, detailed ruins of skyscrapers and infrastructure covered in vegetation, moody and dramatic sky with dynamic lighting, high-resolution dystopian art with intricate textures, hauntingly beautiful and evocative scenery, ultra-realistic details capturing the contrast between nature and decay, space for descriptive text",
        "celestial goddess figure in 8K HD, surrounded by a vast expanse of stars and swirling galaxies, flowing and ethereal garments with intricate cosmic patterns, vibrant and luminous celestial colors enhancing divine presence, high-detail fantasy portrait with rich textures, majestic and powerful aura, ultra-realistic rendering with depth, space for event title and information",
        "dynamic sports action scene in 8K HD, high-energy movement capturing a detailed athlete in motion, vibrant and blurred background emphasizing speed and intensity, ultra-realistic textures and dynamic lighting enhancing the action, intense and captivating composition with depth, high-resolution sports illustration with rich details, space for event details or titles",
        "steampunk airship floating above fluffy clouds in 8K HD, highly detailed Victorian-inspired design with ornate machinery and propellers, warm and metallic color palette with intricate textures, dynamic and majestic pose capturing movement, high-resolution fantasy adventure art with depth, grandiose and awe-inspiring atmosphere, ultra-realistic rendering, space for descriptive text",
        "abstract digital landscape in 8K HD, flowing and dynamic shapes with vibrant and harmonious colors, futuristic and ethereal feel with layered depth, high-resolution and ultra-detailed digital art, visually mesmerizing and immersive design, vibrant and dynamic composition with intricate elements, modern and contemporary aesthetic, space for personalized text",
        "mythical phoenix rising from ashes in 8K HD, vibrant and fiery flames engulfing the scene, detailed and majestic feathers with dynamic fiery colors, dramatic and powerful pose symbolizing rebirth, high-resolution fantasy art with rich textures, symbolic and inspiring imagery with depth, ultra-realistic rendering, space for event information",
        "urban graffiti style art in 8K HD, vibrant and bold spray paint colors creating dynamic and edgy designs, high-resolution street art-inspired illustration with intricate details, expressive and modern composition with depth, ultra-realistic textures enhancing the urban vibe, bold and youthful aesthetic, space for personalized text or titles",
        "futuristic robotics lab in 8K HD, highly detailed robots and advanced AI interfaces interacting, sleek and high-tech environment with glowing screens and holograms, vibrant blue and silver color scheme enhancing modern feel, high-resolution sci-fi illustration with depth, ultra-realistic textures and lighting, space for event details or information",
        "enchanted mountain landscape in 8K HD, towering and majestic peaks with a mystical glow under a vibrant sky, detailed and lush flora and fauna enhancing the natural beauty, serene and magical atmosphere with soft lighting and shadows, high-resolution fantasy scenery with rich textures, vibrant and captivating colors with depth, ultra-realistic rendering, space for descriptive text",
        "abstract light trails in 8K HD, dynamic and fluid movement with colorful streaks creating a sense of motion, high-resolution digital art with vibrant and saturated colors, visually striking and energetic composition with depth, modern and futuristic aesthetic with intricate details, ultra-realistic lighting effects, space for event information",
        "futuristic fashion design in 8K HD, high-tech garments with glowing accents and intricate textures, detailed and innovative materials showcasing advanced fashion technology, vibrant and dynamic colors enhancing modern style, high-resolution fashion illustration with depth and realism, sleek and avant-garde aesthetic, space for personalized text or titles",
        "fantasy underwater palace in 8K HD, intricate and ornate coral architecture illuminated by bioluminescent lights, glowing and vibrant sea creatures swimming gracefully, detailed and magical environment with rich textures, high-resolution fantasy art with depth, enchanting and majestic atmosphere, ultra-realistic rendering, space for event details",
        "cosmic nebula with vibrant colors in 8K HD, highly detailed star clusters and swirling galaxies creating an expansive view, high-resolution space art with rich and saturated colors, visually stunning and ethereal composition with depth, ultra-realistic textures enhancing cosmic beauty, mesmerizing and awe-inspiring scene, space for descriptive text",
        "abstract fractal patterns in 8K HD, highly intricate and symmetrical designs with vibrant and contrasting colors, high-resolution digital art with complex and mesmerizing structures, visually captivating and intellectually stimulating composition with depth, ultra-realistic rendering enhancing detail, modern and contemporary aesthetic, space for personalized titles or information",
        "futuristic transportation system in 8K HD, detailed and sleek hover vehicles gliding on advanced tracks, vibrant and neon-lit city background enhancing modern feel, high-resolution sci-fi illustration with dynamic movement, innovative and cutting-edge design with rich textures, ultra-realistic lighting and shadows, space for event details or information",
        "vintage celestial map in 8K HD, intricate and detailed star constellations with antique color palette, highly detailed celestial symbols and annotations, high-resolution historical illustration with rich textures, elegant and timeless design with depth, ultra-realistic rendering enhancing vintage aesthetics, space for descriptive text or titles",
        "high-tech virtual reality scene in 8K HD, immersive and expansive digital environment with vibrant holographic elements, futuristic VR gear with intricate designs, detailed and expansive virtual landscapes with rich textures, dynamic and interactive feel with depth, high-resolution sci-fi art with ultra-realistic rendering, space for event information",
        "medieval fantasy marketplace in 8K HD, bustling with detailed and diverse characters interacting, vibrant and colorful stalls showcasing exotic goods, intricate architectural elements with medieval design, lively and dynamic atmosphere with rich textures, high-resolution fantasy illustration with depth, ultra-realistic rendering enhancing detail, space for descriptive text or titles",
        "bioluminescent jungle at night in 8K HD, glowing and vibrant plants illuminating the dark environment, detailed and magical creatures roaming the lush flora, misty and ethereal atmosphere with soft lighting, high-resolution fantasy landscape with rich textures, enchanting and mystical ambiance with depth, ultra-realistic rendering, space for event details",
        "art deco city skyline in 8K HD, intricate geometric patterns and vibrant sunset colors reflecting on detailed skyscrapers, high-resolution architectural illustration with ornate decorations, elegant and stylish design with rich textures, ultra-realistic rendering enhancing depth and detail, visually striking and timeless aesthetic, space for descriptive text or titles",
        "futuristic underwater exploration vessel in 8K HD, highly detailed and sleek design with glowing lights, surrounded by vibrant and diverse marine life, advanced and innovative technology integrated into the vessel, high-resolution sci-fi art with rich textures, ultra-realistic rendering enhancing depth, space for event information or titles",
        "mythological gods in 8K HD, intricate and divine attire with vibrant celestial backgrounds, detailed expressions and dynamic poses showcasing power and majesty, high-resolution fantasy portrait with rich textures, majestic and powerful presence with depth, ultra-realistic rendering enhancing divine elements, space for descriptive text or titles",
        "neon-lit futuristic nightclub in 8K HD, vibrant and dynamic lighting with glowing holograms, detailed and modern interior design with sleek furniture, diverse and futuristic characters enjoying the lively atmosphere, high-resolution digital art with rich textures, ultra-realistic rendering enhancing vibrant colors, space for event details or information",
        "fantasy elven city in 8K HD, intricate and ornate treehouse architecture nestled among vibrant natural surroundings, detailed and graceful elven characters interacting, magical and serene atmosphere with soft lighting, high-resolution fantasy landscape with rich textures, enchanting and captivating environment with depth, ultra-realistic rendering, space for descriptive text or titles",
        "robotic wildlife in 8K HD, highly detailed and realistic mechanical animals blending seamlessly with vibrant natural backgrounds, fusion of advanced technology and organic elements creating a unique aesthetic, high-resolution sci-fi art with rich textures, innovative and visually stunning composition with depth, ultra-realistic rendering enhancing detail, space for event information or titles",
        "galactic battle scene in 8K HD, highly detailed and dynamic spaceships engaged in intense laser warfare, vibrant cosmic background with swirling galaxies and explosive effects, high-resolution sci-fi illustration with rich textures, dynamic and action-packed composition with depth, ultra-realistic rendering enhancing dramatic elements, space for descriptive text or titles",
        "enchanted fairy garden in 8K HD, vibrant and colorful flowers glowing softly under magical lighting, intricate and whimsical garden details with fluttering fairies, serene and whimsical atmosphere with rich textures, high-resolution fantasy art with depth, enchanting and magical ambiance with ultra-realistic rendering, space for event details or information",
        "futuristic sports arena in 8K HD, highly detailed and high-tech design with vibrant lighting and holographic displays, dynamic and action-packed scenes capturing athletes in motion, rich and detailed textures enhancing the modern feel, high-resolution sci-fi illustration with depth, ultra-realistic rendering enhancing vibrant colors, space for descriptive text or titles",
        "mythical underwater creatures in 8K HD, highly detailed and vibrant marine life with enchanting and magical features, intricate and colorful textures enhancing the fantastical elements, serene and mystical underwater atmosphere with soft lighting, high-resolution fantasy art with depth, ultra-realistic rendering capturing intricate details, space for event information or titles",
        "post-modern abstract sculpture in 8K HD, highly intricate and geometric forms with vibrant and contrasting color palette, detailed and textured materials enhancing the modern design, high-resolution digital art with rich textures, visually striking and innovative composition with depth, ultra-realistic rendering enhancing abstract elements, space for descriptive text or titles",
        "futuristic drone fleet in 8K HD, highly detailed and sleek drone designs flying in coordinated formations, vibrant aerial background with dynamic movement and light trails, high-resolution sci-fi illustration with rich textures, dynamic and innovative composition with depth, ultra-realistic rendering enhancing technological details, space for event information or titles",
        "celestial dance of planets in 8K HD, vibrant and swirling cosmic colors with detailed planetary textures, dynamic and harmonious composition showcasing orbital movements, high-resolution space art with rich textures, ethereal and mesmerizing ambiance with depth, ultra-realistic rendering enhancing celestial beauty, space for descriptive text or titles",
        "fantasy warrior in 8K HD, highly detailed and intricate armor with vibrant and dynamic background, powerful and heroic pose capturing strength and determination, detailed expressions and rich textures enhancing realism, high-resolution fantasy portrait with depth, ultra-realistic rendering showcasing heroic elements, space for event information or titles",
        "virtual metropolis at night in 8K HD, vibrant and neon-lit cityscape with detailed futuristic buildings and bustling digital streets, high-resolution sci-fi art with rich textures, dynamic and lively atmosphere with depth, ultra-realistic rendering enhancing vibrant colors and lighting, space for descriptive text or titles"
    ]


    app = QApplication(sys.argv)
    window = MainWindow(prompts)
    window.show()
    sys.exit(app.exec_())

NameError: name 'ReplicateClient' is not defined

In [3]:
import sys
import random
import os
import replicate
from replicate.client import Client
import requests
import logging  # Import logging module
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QLabel,
    QScrollArea, QHBoxLayout, QMessageBox, QGridLayout, QMainWindow, QFrame,
    QTabWidget, QTextEdit, QSplitter
)
from PyQt5.QtCore import Qt, QTimer, QPropertyAnimation, QRect, pyqtProperty, QSize, QThread, pyqtSignal
from PyQt5.QtGui import QClipboard, QPixmap, QPainter, QIcon, QTransform, QColor, QFont, QPen
from PyQt5.QtWidgets import QGraphicsOpacityEffect
from PyQt5.QtWidgets import QDialog, QVBoxLayout, QPushButton, QTextEdit, QLabel, QHBoxLayout, QMessageBox
from PyQt5.QtCore import pyqtSignal, Qt
from openai import OpenAI  # Import OpenAI

# Initialize OpenAI client
openai_client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Initialize Replicate client
replicate_client = Client(api_token=os.environ['REPLICATE_API_TOKEN'])

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Check for API keys
if not os.environ.get('REPLICATE_API_TOKEN'):
    logging.error("REPLICATE_API_TOKEN is not set.")
    raise SystemExit("Error: REPLICATE_API_TOKEN is not set.")

if not os.environ.get('OPENAI_API_KEY'):
    logging.error("OPENAI_API_KEY is not set.")
    raise SystemExit("Error: OPENAI_API_KEY is not set.")
# Set up OpenAI API key
client.api_key = OPENAI_API_KEY

try:
    # Try to run a simple model to test the connection
    client.run("black-forest-labs/flux-pro", input={"prompt": "test"})
    logging.info("Successfully connected to Replicate")
except Exception as e:
    logging.error(f"Error connecting to Replicate: {str(e)}")
    # Instead of sys.exit(1), we'll just log the error
    logging.error("Failed to connect to Replicate. The application may not function correctly.")


# Toast Notification Class
class Toast(QWidget):
    def __init__(self, message, parent=None):
        super().__init__(parent)
        self.setWindowFlags(Qt.FramelessWindowHint | Qt.WindowStaysOnTopHint | Qt.Tool)
        self.setAttribute(Qt.WA_TranslucentBackground)
        self.opacity_effect = QGraphicsOpacityEffect(self)
        self.setGraphicsEffect(self.opacity_effect)
        self.opacity_effect.setOpacity(0)
        self.label = QLabel(message, self)
        self.label.setStyleSheet("""
            QLabel {
                background-color: rgba(0, 0, 0, 180);
                color: white;
                border-radius: 10px;
                padding: 10px;
            }
        """)
        font = QFont()
        font.setPointSize(10)
        self.label.setFont(font)
        self.animation = QPropertyAnimation(self.opacity_effect, b"opacity")
        self.animation.setDuration(2000)
        self.animation.setStartValue(0)
        self.animation.setKeyValueAt(0.1, 1)
        self.animation.setKeyValueAt(0.9, 1)
        self.animation.setEndValue(0)
        self.animation.finished.connect(self.close)

    def show_toast(self):
        self.show()
        self.animation.start()

# Worker Thread for Image Generation
class ImageGeneratorThread(QThread):
    finished = pyqtSignal(str)  # Signal to emit the image URL

    def __init__(self, prompt):
        super().__init__()
        self.prompt = prompt

    def run(self):
        try:
            # Use the global replicate_client
            output = replicate_client.run(
                "black-forest-labs/flux-pro",
                input={"prompt": self.prompt}
            )
            # Since the output is a direct URL, emit it
            self.finished.emit(output)
        except Exception as e:
            self.finished.emit(f"Error: {str(e)}")

# Worker Thread for Prompt Generation

class PromptGeneratorThread(QThread):
    finished = pyqtSignal(str)

    def run(self):
        try:
            system_prompt = (
                "You are a creative AI assistant specializing in generating image prompts. "
                "Generate a creative and detailed image prompt for an AI image generation model. "
                "The prompt should be similar in style and detail to high-quality prompts that include words like "
                "'High quality', 'hd', or '8k'. The prompt should be imaginative, descriptive, and suitable for generating "
                "a visually stunning image."
            )
            response = openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": "Generate an image prompt."}
                ],
                max_tokens=200,
                n=1,
                temperature=0.8,
            )
            generated_prompt = response.choices[0].message.content.strip()
            self.finished.emit(generated_prompt)
        except Exception as e:
            self.finished.emit(f"Error: {str(e)}")
            
# Custom Popup Dialog
class PromptPopup(QWidget):
    def __init__(self, prompt_text, parent=None):
        super().__init__(parent)
        self.setWindowFlags(Qt.Popup | Qt.FramelessWindowHint)
        self.setAttribute(Qt.WA_TranslucentBackground)
        self.prompt_text = prompt_text

        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout(self)
        layout.setContentsMargins(20, 20, 20, 20)

        label = QLabel(self.prompt_text)
        label.setStyleSheet("color: white;")
        label.setWordWrap(True)
        layout.addWidget(label)

        self.setStyleSheet("""
            QWidget {
                background-color: black;
                border: 2px solid white;
                border-radius: 15px;
            }
        """)
        self.adjustSize()

    def mousePressEvent(self, event):
        event.accept()
        self.close()

# Custom Rounded Label
class RoundedClickableLabel(QLabel):
    clicked = pyqtSignal()

    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setStyleSheet("""
            QLabel {
                background-color: #000000;
                border: 2px solid #ffffff;
                border-radius: 15px;
                padding: 15px;
                color: #ffffff;
            }
            QLabel:hover {
                background-color: #333333;
            }
        """)
        self.setWordWrap(True)
        self.setCursor(Qt.PointingHandCursor)
        self.setFont(QFont("Arial", 10))

    def mousePressEvent(self, event):
        self.clicked.emit()

# Spinning Button
class SpinningButton(QPushButton):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setFixedSize(150, 60)
        self.setStyleSheet("""
            QPushButton {
                background-color: #4CAF50;
                border-radius: 30px;
                color: white;
                font-weight: bold;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
        """)
        self.spinning = False
        self.angle = 0
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_rotation)
        self.original_text = text

    def paintEvent(self, event):
        super().paintEvent(event)
        if self.spinning:
            painter = QPainter(self)
            painter.setRenderHint(QPainter.Antialiasing)
            painter.translate(self.width() / 2, self.height() / 2)
            painter.rotate(self.angle)
            # Draw a reload symbol
            pen = QPen(Qt.white, 3)
            painter.setPen(pen)
            painter.drawArc(-12, -12, 24, 24, 30 * 16, 300 * 16)
            painter.drawLine(0, -12, 6, -12)
            painter.drawLine(6, -12, 6, -6)

    def start_spinning(self):
        self.spinning = True
        self.setText("")
        self.timer.start(50)  # Update every 50ms

    def stop_spinning(self):
        self.spinning = False
        self.setText(self.original_text)
        self.timer.stop()

    def update_rotation(self):
        self.angle += 10
        if self.angle >= 360:
            self.angle = 0
        self.update()

# Main Application Window
class MainWindow(QMainWindow):
    def __init__(self, prompts):
        super().__init__()
        self.setWindowTitle("AI Society Showcase")
        self.setGeometry(100, 100, 1000, 800)
        self.prompts = prompts

        # Initialize Clipboard
        self.clipboard = QApplication.clipboard()

        # Central Widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # Tabs
        self.tabs = QTabWidget()
        main_layout.addWidget(self.tabs)

        self.prompts_tab = QWidget()
        self.image_tab = QWidget()

        self.tabs.addTab(self.prompts_tab, "Prompts")
        self.tabs.addTab(self.image_tab, "Image Generation")

        self.init_prompts_tab()
        self.init_image_tab()

        # Set dark theme
        self.setStyleSheet("""
            QMainWindow {
                background-color: #000000;
            }
            QLabel {
                color: #ffffff;
            }
            QPushButton {
                background-color: #333333;
                color: #ffffff;
            }
            QTextEdit {
                background-color: #222222;
                color: #ffffff;
            }
            QTabWidget::pane {
                border: 1px solid #444444;
            }
            QTabBar::tab {
                background: #333333;
                color: #ffffff;
                padding: 10px;
            }
            QTabBar::tab:selected {
                background: #555555;
            }
            QScrollArea {
                background-color: #000000;
            }
        """)

    def init_prompts_tab(self):
        layout = QVBoxLayout(self.prompts_tab)

        # Add randomize and create buttons
        self.randomize_button = SpinningButton("Randomize")
        self.randomize_button.clicked.connect(self.randomize_prompt)
        self.randomize_button.original_text = "Randomize"

        self.create_button = SpinningButton("Create")
        self.create_button.setStyleSheet("""
            QPushButton {
                background-color: #2196F3;
                border-radius: 30px;
                color: white;
                font-weight: bold;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #1976D2;
            }
        """)
        self.create_button.clicked.connect(self.create_prompt)
        self.create_button.original_text = "Create"

        button_layout = QHBoxLayout()
        button_layout.addWidget(self.randomize_button)
        button_layout.addWidget(self.create_button)
        layout.addLayout(button_layout)

        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        layout.addWidget(scroll_area)

        container = QWidget()
        scroll_area.setWidget(container)
        self.container_layout = QVBoxLayout(container)
        self.container_layout.setAlignment(Qt.AlignTop)

        self.update_prompts_list()

    def update_prompts_list(self):
        # Clear existing prompts
        for i in reversed(range(self.container_layout.count())):
            widget = self.container_layout.takeAt(i).widget()
            if widget:
                widget.deleteLater()

        for prompt in self.prompts:
            prompt_label = RoundedClickableLabel(prompt)
            prompt_label.clicked.connect(lambda checked, p=prompt: self.copy_to_clipboard(p))
            self.container_layout.addWidget(prompt_label)
            self.container_layout.addSpacing(10)

    def randomize_prompt(self):
        if self.prompts:
            self.randomize_button.start_spinning()
            QTimer.singleShot(1000, self.finish_randomize)  # 1 second of spinning

    def finish_randomize(self):
        self.randomize_button.stop_spinning()
        random_prompt = random.choice(self.prompts)
        self.copy_to_clipboard(random_prompt)
        self.show_toast("Random prompt copied to clipboard!")

    def create_prompt(self):
        self.create_button.start_spinning()
        # Open the prompt input dialog
        self.prompt_input_dialog = PromptInputDialog(self)
        self.prompt_input_dialog.prompt_entered.connect(self.insert_prompt_to_generation_tab)
        self.prompt_input_dialog.show()

    def insert_prompt_to_generation_tab(self, prompt):
        # Switch to image generation tab
        self.tabs.setCurrentWidget(self.image_tab)
        # Paste the prompt into the prompt edit area
        self.prompt_edit.setText(prompt)
        # Optionally, show a toast notification
        self.show_toast("Prompt ready for image generation!")

    def init_image_tab(self):
        layout = QVBoxLayout(self.image_tab)

        # Splitter to adjust sizes
        splitter = QSplitter(Qt.Vertical)
        layout.addWidget(splitter)

        # Image Display Label (larger portion)
        self.image_label = QLabel("Your generated image will appear here.")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("background-color: #111111; color: #ffffff;")
        self.image_label.setMinimumHeight(400)
        splitter.addWidget(self.image_label)

        # Prompt Input Area (smaller portion)
        prompt_container = QWidget()
        prompt_layout = QVBoxLayout(prompt_container)
        prompt_layout.setContentsMargins(10, 10, 10, 10)

        self.prompt_edit = QTextEdit()
        self.prompt_edit.setPlaceholderText("Enter your prompt here")
        self.prompt_edit.setFixedHeight(100)
        prompt_layout.addWidget(self.prompt_edit)

        self.generate_button = QPushButton("Generate Image")
        self.generate_button.clicked.connect(self.on_generate_clicked)
        prompt_layout.addWidget(self.generate_button)

        splitter.addWidget(prompt_container)

        splitter.setStretchFactor(0, 3)  # Image label takes 3 parts
        splitter.setStretchFactor(1, 1)  # Prompt input takes 1 part

    def on_generate_clicked(self):
        prompt = self.prompt_edit.toPlainText().strip()
        if not prompt:
            QMessageBox.warning(self, "Warning", "Please enter a prompt.")
            return

        # Disable the button to prevent multiple clicks
        self.generate_button.setEnabled(False)
        self.image_label.setText("Generating Image...")

        # Start the image generation in a separate thread
        self.image_thread = ImageGeneratorThread(prompt)
        self.image_thread.finished.connect(self.on_image_generated)
        self.image_thread.start()

    def copy_to_clipboard(self, text):
        self.clipboard.setText(text)
        self.show_toast("Prompt copied to clipboard!")
        # Automatically switch to the Image Generation tab
        self.tabs.setCurrentWidget(self.image_tab)
        self.prompt_edit.setText(text)

    def show_toast(self, message):
        toast = Toast(message, self)
        toast.move(self.width() // 2 - toast.width() // 2, 50)
        toast.show_toast()

    def on_image_generated(self, output):
        self.generate_button.setEnabled(True)
        if output.startswith("Error"):
            self.image_label.setText("Failed to generate image.")
            self.show_toast(output)
        else:
            # Download the image and display it
            try:
                response = requests.get(output)
                if response.status_code == 200:
                    image = QPixmap()
                    image.loadFromData(response.content)
                    scaled_image = image.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    self.image_label.setPixmap(scaled_image)
                    self.image_label.setText("")
                else:
                    self.image_label.setText("Failed to load image.")
                    self.show_toast("Failed to load generated image.")
            except Exception as e:
                self.image_label.setText("Error loading image.")
                self.show_toast(str(e))

class PromptInputDialog(QDialog):
    prompt_entered = pyqtSignal(str)

    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Enter Prompt")
        self.setModal(True)
        self.setWindowFlags(self.windowFlags() | Qt.WindowStaysOnTopHint)
        self.setStyleSheet("""
            QDialog {
                background-color: #000000;
            }
            QLabel, QTextEdit {
                color: #FFFFFF;
            }
            QPushButton {
                background-color: #2196F3;
                color: #FFFFFF;
                padding: 10px;
                border-radius: 5px;
            }
            QPushButton:hover {
                background-color: #1976D2;
            }
        """)

        layout = QVBoxLayout(self)

        self.prompt_label = QLabel("Enter your prompt:")
        layout.addWidget(self.prompt_label)

        self.prompt_edit = QTextEdit()
        self.prompt_edit.setFixedHeight(100)
        layout.addWidget(self.prompt_edit)

        button_layout = QHBoxLayout()

        self.generate_ai_button = QPushButton("Generate with AI")
        self.generate_ai_button.clicked.connect(self.generate_prompt_with_ai)
        button_layout.addWidget(self.generate_ai_button)

        self.ok_button = QPushButton("OK")
        self.ok_button.clicked.connect(self.on_ok_clicked)
        button_layout.addWidget(self.ok_button)

        layout.addLayout(button_layout)

    def generate_prompt_with_ai(self):
        self.generate_ai_button.setEnabled(False)
        self.generate_ai_button.setText("Generating...")

        # Start the prompt generation in a separate thread
        self.prompt_thread = PromptGeneratorThread()
        self.prompt_thread.finished.connect(self.on_prompt_generated)
        self.prompt_thread.start()

    def on_prompt_generated(self, generated_prompt):
        self.generate_ai_button.setEnabled(True)
        self.generate_ai_button.setText("Generate with AI")

        if generated_prompt.startswith("Error"):
            QMessageBox.warning(self, "Error", f"Failed to generate prompt:\n{generated_prompt}")
        else:
            # Insert the generated prompt into the input field
            self.prompt_edit.setText(generated_prompt)

    def on_ok_clicked(self):
        prompt = self.prompt_edit.toPlainText().strip()
        if prompt:
            self.prompt_entered.emit(prompt)
            self.accept()  # Close the dialog
        else:
            QMessageBox.warning(self, "Warning", "Please enter a prompt or generate one with AI.")
# Entry Point
if __name__ == "__main__":
    # Ensure the Replicate API token is set
    if not REPLICATE_API_TOKEN:
        print("Error: REPLICATE_API_TOKEN is not set.")
        sys.exit(1)

    prompts = [
        "futuristic 8K HD poster design, AI Showcase event, sleek modern cityscape background, glowing neon accents, circuit patterns, large abstract AI brain foreground, bright blue purple pink lights, intricate interconnected nodes and wires, floating holographic elements, streams of digital data, 3D geometric shapes, dynamic high-tech atmosphere, clean modern vibrant style, sharp detailed features, space at top for event text information, high quality, ultra-realistic rendering, immersive depth and perspective",
        "surreal 8K HD dreamscape, vibrant and contrasting colors, floating islands with lush greenery and cascading waterfalls, whimsical and mythical creatures interacting, twilight sky filled with twinkling stars and colorful nebulae, reflective and crystal-clear lakes, intricate and ornate floral patterns, soft glowing ambient lights, magical and enchanting atmosphere, high-resolution fantasy art, ultra-detailed textures, immersive and otherworldly environment, space for personalized text",
        "steampunk-inspired 8K HD machinery scene, highly detailed gears and cogs in motion, vintage brass and copper color palette, ornate and intricate metalwork, Victorian-era aesthetics with modern twists, steam-powered robots performing tasks, complex industrial background with pipes and steam vents, sepia tones with vibrant highlights, high-detail illustration with depth, dynamic and action-packed composition, ultra-realistic textures, space for event information",
        "abstract geometric 8K HD patterns, bold and vibrant colors with high contrast, symmetrical and intricate designs, interlocking and overlapping shapes creating depth, 3D effects with shadows and highlights, modern and contemporary art style, smooth vibrant gradients blending seamlessly, precise and intricate line work, high-resolution digital art with sharp clarity, visually striking and balanced composition, ultra-detailed elements, space for custom text overlays",
        "cyberpunk 8K HD street scene, neon-lit alleyways with vibrant signage, rainy weather creating reflective surfaces, diverse and futuristic characters with unique styles, holographic advertisements floating in the air, high-tech gadgets and wearable tech accessories, dark and moody atmosphere with contrasting bright lights, detailed urban environment with towering skyscrapers, vibrant color contrasts with electric blues and hot pinks, ultra-realistic textures, space for event details",
        "fantasy dragon 8K HD soaring over a medieval castle, dramatic and dynamic sky with sunset hues blending into twilight, highly intricate dragon scales shimmering with vibrant colors, detailed and majestic castle architecture with towering spires, dynamic action pose capturing movement, rich and textured environments with lush landscapes, high-resolution fantasy illustration with depth, epic and majestic feel, space for descriptive text",
        "underwater utopia in 8K HD, vibrant and colorful coral reefs teeming with diverse marine life, sunlight filtering through crystal-clear water creating dynamic light patterns, futuristic underwater cities with sleek architecture, bioluminescent plants and creatures glowing softly, serene and peaceful atmosphere with gentle currents, high-detail aquatic scene with realistic textures, vivid and saturated colors, ultra-realistic rendering, space for informational text",
        "space exploration 8K HD theme, highly detailed spaceship orbiting a distant and vibrant planet, starry background with vivid galaxies and nebulae, astronaut in a dynamic pose floating in zero gravity, vibrant celestial colors with rich textures, high-resolution sci-fi art with expansive depth, awe-inspiring and immersive view, ultra-realistic space elements, space for event title and details",
        "ancient mystical forest in 8K HD, towering and ancient trees with glowing bioluminescent leaves, magical and mythical creatures roaming the forest floor, misty and ethereal atmosphere with soft light beams piercing through the canopy, intricate details in flora and fauna with rich textures, serene and enchanting ambiance, high-resolution fantasy landscape with depth, ultra-realistic rendering, space for descriptive text",
        "modern minimalist 8K HD design, clean and sharp lines with precise geometry, monochromatic color scheme accented with a vibrant pop of color, abstract representation of advanced technology, high-resolution digital art with sleek and sophisticated aesthetic, spacious and uncluttered composition, ultra-detailed elements with subtle gradients, visually appealing and elegant, space for event information",
        "retro 80s synthwave 8K HD poster, vibrant and glowing neon colors with high saturation, grid landscape fading into a colorful sunset, futuristic and sleek retro cars speeding along the grid, pixelated and digital elements enhancing the nostalgic vibe, dynamic and energetic composition with motion effects, high-detail digital illustration with depth, stylish and eye-catching design, space for event title and details",
        "abstract expressionism 8K HD artwork, bold and dynamic brushstrokes with vibrant and contrasting colors, emotional and energetic composition conveying movement, high-resolution fine art style with rich textures, visually impactful and thought-provoking elements, ultra-detailed layers creating depth, modern and contemporary aesthetic, space for artistic text or titles",
        "biotech-inspired 8K HD visuals, DNA strands intricately intertwined with glowing circuitry, vibrant and pulsating light elements enhancing the futuristic feel, high-tech laboratory background with advanced equipment, fusion of organic and technological motifs, high-resolution and ultra-detailed design, vibrant and dynamic color palette, visually stunning and innovative, space for descriptive text",
        "magical library interior in 8K HD, endless rows of ancient bookshelves with floating and glowing books, warm and ambient lighting casting soft shadows, intricate architectural details with ornate decorations, whimsical and cozy atmosphere inviting exploration, high-resolution fantasy illustration with rich textures, enchanting and inviting environment, ultra-realistic rendering, space for event information",
        "post-apocalyptic 8K HD landscape, abandoned and decaying city overtaken by lush and wild nature, detailed ruins of skyscrapers and infrastructure covered in vegetation, moody and dramatic sky with dynamic lighting, high-resolution dystopian art with intricate textures, hauntingly beautiful and evocative scenery, ultra-realistic details capturing the contrast between nature and decay, space for descriptive text",
        "celestial goddess figure in 8K HD, surrounded by a vast expanse of stars and swirling galaxies, flowing and ethereal garments with intricate cosmic patterns, vibrant and luminous celestial colors enhancing divine presence, high-detail fantasy portrait with rich textures, majestic and powerful aura, ultra-realistic rendering with depth, space for event title and information",
        "dynamic sports action scene in 8K HD, high-energy movement capturing a detailed athlete in motion, vibrant and blurred background emphasizing speed and intensity, ultra-realistic textures and dynamic lighting enhancing the action, intense and captivating composition with depth, high-resolution sports illustration with rich details, space for event details or titles",
        "steampunk airship floating above fluffy clouds in 8K HD, highly detailed Victorian-inspired design with ornate machinery and propellers, warm and metallic color palette with intricate textures, dynamic and majestic pose capturing movement, high-resolution fantasy adventure art with depth, grandiose and awe-inspiring atmosphere, ultra-realistic rendering, space for descriptive text",
        "abstract digital landscape in 8K HD, flowing and dynamic shapes with vibrant and harmonious colors, futuristic and ethereal feel with layered depth, high-resolution and ultra-detailed digital art, visually mesmerizing and immersive design, vibrant and dynamic composition with intricate elements, modern and contemporary aesthetic, space for personalized text",
        "mythical phoenix rising from ashes in 8K HD, vibrant and fiery flames engulfing the scene, detailed and majestic feathers with dynamic fiery colors, dramatic and powerful pose symbolizing rebirth, high-resolution fantasy art with rich textures, symbolic and inspiring imagery with depth, ultra-realistic rendering, space for event information",
        "urban graffiti style art in 8K HD, vibrant and bold spray paint colors creating dynamic and edgy designs, high-resolution street art-inspired illustration with intricate details, expressive and modern composition with depth, ultra-realistic textures enhancing the urban vibe, bold and youthful aesthetic, space for personalized text or titles",
        "futuristic robotics lab in 8K HD, highly detailed robots and advanced AI interfaces interacting, sleek and high-tech environment with glowing screens and holograms, vibrant blue and silver color scheme enhancing modern feel, high-resolution sci-fi illustration with depth, ultra-realistic textures and lighting, space for event details or information",
        "enchanted mountain landscape in 8K HD, towering and majestic peaks with a mystical glow under a vibrant sky, detailed and lush flora and fauna enhancing the natural beauty, serene and magical atmosphere with soft lighting and shadows, high-resolution fantasy scenery with rich textures, vibrant and captivating colors with depth, ultra-realistic rendering, space for descriptive text",
        "abstract light trails in 8K HD, dynamic and fluid movement with colorful streaks creating a sense of motion, high-resolution digital art with vibrant and saturated colors, visually striking and energetic composition with depth, modern and futuristic aesthetic with intricate details, ultra-realistic lighting effects, space for event information",
        "futuristic fashion design in 8K HD, high-tech garments with glowing accents and intricate textures, detailed and innovative materials showcasing advanced fashion technology, vibrant and dynamic colors enhancing modern style, high-resolution fashion illustration with depth and realism, sleek and avant-garde aesthetic, space for personalized text or titles",
        "fantasy underwater palace in 8K HD, intricate and ornate coral architecture illuminated by bioluminescent lights, glowing and vibrant sea creatures swimming gracefully, detailed and magical environment with rich textures, high-resolution fantasy art with depth, enchanting and majestic atmosphere, ultra-realistic rendering, space for event details",
        "cosmic nebula with vibrant colors in 8K HD, highly detailed star clusters and swirling galaxies creating an expansive view, high-resolution space art with rich and saturated colors, visually stunning and ethereal composition with depth, ultra-realistic textures enhancing cosmic beauty, mesmerizing and awe-inspiring scene, space for descriptive text",
        "abstract fractal patterns in 8K HD, highly intricate and symmetrical designs with vibrant and contrasting colors, high-resolution digital art with complex and mesmerizing structures, visually captivating and intellectually stimulating composition with depth, ultra-realistic rendering enhancing detail, modern and contemporary aesthetic, space for personalized titles or information",
        "futuristic transportation system in 8K HD, detailed and sleek hover vehicles gliding on advanced tracks, vibrant and neon-lit city background enhancing modern feel, high-resolution sci-fi illustration with dynamic movement, innovative and cutting-edge design with rich textures, ultra-realistic lighting and shadows, space for event details or information",
        "vintage celestial map in 8K HD, intricate and detailed star constellations with antique color palette, highly detailed celestial symbols and annotations, high-resolution historical illustration with rich textures, elegant and timeless design with depth, ultra-realistic rendering enhancing vintage aesthetics, space for descriptive text or titles",
        "high-tech virtual reality scene in 8K HD, immersive and expansive digital environment with vibrant holographic elements, futuristic VR gear with intricate designs, detailed and expansive virtual landscapes with rich textures, dynamic and interactive feel with depth, high-resolution sci-fi art with ultra-realistic rendering, space for event information",
        "medieval fantasy marketplace in 8K HD, bustling with detailed and diverse characters interacting, vibrant and colorful stalls showcasing exotic goods, intricate architectural elements with medieval design, lively and dynamic atmosphere with rich textures, high-resolution fantasy illustration with depth, ultra-realistic rendering enhancing detail, space for descriptive text or titles",
        "bioluminescent jungle at night in 8K HD, glowing and vibrant plants illuminating the dark environment, detailed and magical creatures roaming the lush flora, misty and ethereal atmosphere with soft lighting, high-resolution fantasy landscape with rich textures, enchanting and mystical ambiance with depth, ultra-realistic rendering, space for event details",
        "art deco city skyline in 8K HD, intricate geometric patterns and vibrant sunset colors reflecting on detailed skyscrapers, high-resolution architectural illustration with ornate decorations, elegant and stylish design with rich textures, ultra-realistic rendering enhancing depth and detail, visually striking and timeless aesthetic, space for descriptive text or titles",
        "futuristic underwater exploration vessel in 8K HD, highly detailed and sleek design with glowing lights, surrounded by vibrant and diverse marine life, advanced and innovative technology integrated into the vessel, high-resolution sci-fi art with rich textures, ultra-realistic rendering enhancing depth, space for event information or titles",
        "mythological gods in 8K HD, intricate and divine attire with vibrant celestial backgrounds, detailed expressions and dynamic poses showcasing power and majesty, high-resolution fantasy portrait with rich textures, majestic and powerful presence with depth, ultra-realistic rendering enhancing divine elements, space for descriptive text or titles",
        "neon-lit futuristic nightclub in 8K HD, vibrant and dynamic lighting with glowing holograms, detailed and modern interior design with sleek furniture, diverse and futuristic characters enjoying the lively atmosphere, high-resolution digital art with rich textures, ultra-realistic rendering enhancing vibrant colors, space for event details or information",
        "fantasy elven city in 8K HD, intricate and ornate treehouse architecture nestled among vibrant natural surroundings, detailed and graceful elven characters interacting, magical and serene atmosphere with soft lighting, high-resolution fantasy landscape with rich textures, enchanting and captivating environment with depth, ultra-realistic rendering, space for descriptive text or titles",
        "robotic wildlife in 8K HD, highly detailed and realistic mechanical animals blending seamlessly with vibrant natural backgrounds, fusion of advanced technology and organic elements creating a unique aesthetic, high-resolution sci-fi art with rich textures, innovative and visually stunning composition with depth, ultra-realistic rendering enhancing detail, space for event information or titles",
        "galactic battle scene in 8K HD, highly detailed and dynamic spaceships engaged in intense laser warfare, vibrant cosmic background with swirling galaxies and explosive effects, high-resolution sci-fi illustration with rich textures, dynamic and action-packed composition with depth, ultra-realistic rendering enhancing dramatic elements, space for descriptive text or titles",
        "enchanted fairy garden in 8K HD, vibrant and colorful flowers glowing softly under magical lighting, intricate and whimsical garden details with fluttering fairies, serene and whimsical atmosphere with rich textures, high-resolution fantasy art with depth, enchanting and magical ambiance with ultra-realistic rendering, space for event details or information",
        "futuristic sports arena in 8K HD, highly detailed and high-tech design with vibrant lighting and holographic displays, dynamic and action-packed scenes capturing athletes in motion, rich and detailed textures enhancing the modern feel, high-resolution sci-fi illustration with depth, ultra-realistic rendering enhancing vibrant colors, space for descriptive text or titles",
        "mythical underwater creatures in 8K HD, highly detailed and vibrant marine life with enchanting and magical features, intricate and colorful textures enhancing the fantastical elements, serene and mystical underwater atmosphere with soft lighting, high-resolution fantasy art with depth, ultra-realistic rendering capturing intricate details, space for event information or titles",
        "post-modern abstract sculpture in 8K HD, highly intricate and geometric forms with vibrant and contrasting color palette, detailed and textured materials enhancing the modern design, high-resolution digital art with rich textures, visually striking and innovative composition with depth, ultra-realistic rendering enhancing abstract elements, space for descriptive text or titles",
        "futuristic drone fleet in 8K HD, highly detailed and sleek drone designs flying in coordinated formations, vibrant aerial background with dynamic movement and light trails, high-resolution sci-fi illustration with rich textures, dynamic and innovative composition with depth, ultra-realistic rendering enhancing technological details, space for event information or titles",
        "celestial dance of planets in 8K HD, vibrant and swirling cosmic colors with detailed planetary textures, dynamic and harmonious composition showcasing orbital movements, high-resolution space art with rich textures, ethereal and mesmerizing ambiance with depth, ultra-realistic rendering enhancing celestial beauty, space for descriptive text or titles",
        "fantasy warrior in 8K HD, highly detailed and intricate armor with vibrant and dynamic background, powerful and heroic pose capturing strength and determination, detailed expressions and rich textures enhancing realism, high-resolution fantasy portrait with depth, ultra-realistic rendering showcasing heroic elements, space for event information or titles",
        "virtual metropolis at night in 8K HD, vibrant and neon-lit cityscape with detailed futuristic buildings and bustling digital streets, high-resolution sci-fi art with rich textures, dynamic and lively atmosphere with depth, ultra-realistic rendering enhancing vibrant colors and lighting, space for descriptive text or titles"
    ]


    app = QApplication(sys.argv)
    window = MainWindow(prompts)
    window.show()
    sys.exit(app.exec_())

2024-09-19 08:57:17,210 - ERROR - Error connecting to Replicate: 'OpenAI' object has no attribute 'run'
2024-09-19 08:57:17,211 - ERROR - Failed to connect to Replicate. The application may not function correctly.
2024-09-19 08:57:37,552 - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are a creative AI assistant specializing in generating image prompts. Generate a creative and detailed image prompt for an AI image generation model. The prompt should be similar in style and detail to high-quality prompts that include words like 'High quality', 'hd', or '8k'. The prompt should be imaginative, descriptive, and suitable for generating a visually stunning image."}, {'role': 'user', 'content': 'Generate an image prompt.'}], 'model': 'gpt-3.5-turbo', 'max_tokens': 200, 'n': 1, 'temperature': 0.8}}
2024-09-19 08:57:37,554 - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/chat/com

KeyboardInterrupt: 

KeyboardInterrupt: 

SystemExit: 0

c:\Users\22837352\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
